# Train the logistic regression model

In [2]:
import numpy
import pandas as pd

## Import Iris data

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [4]:
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [5]:
X[0]

array([5.1, 3.5, 1.4, 0.2])

## Train the model

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
clr = LogisticRegression()
clr.fit(X_train, y_train)

LogisticRegression()

## Convert model to onnx

Details about conversion of sklearn models to onnx can be found here: https://onnx.ai/sklearn-onnx/introduction.html

In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [8]:
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("logreg_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [9]:
import onnxruntime as rt

In [12]:
sess = rt.InferenceSession("logreg_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

In [14]:
pred_onx[0]

1

In [16]:
import onnx

In [20]:
onnx_model = onnx.load("logreg_iris.onnx")

In [21]:
onnx_model.graph.output

[name: "output_label"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
      }
    }
  }
}
, name: "output_probability"
type {
  sequence_type {
    elem_type {
      map_type {
        key_type: 7
        value_type {
          tensor_type {
            elem_type: 1
          }
        }
      }
    }
  }
}
]

In [22]:
onnx_model.graph.input

[name: "float_input"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
      }
      dim {
        dim_value: 4
      }
    }
  }
}
]

## Load onnx model to Teradata

In [23]:
import teradataml as tdml

In [24]:
import json

In [25]:
with open('teradata.json') as json_file:
    td_conf = json.load(json_file)

tdml.create_context(host=td_conf['host'],
                username=td_conf['user'],
                password=td_conf['password'],
                database=td_conf['schema'])
conn = tdml.get_connection()

/opt/conda/lib/python3.9/site-packages/teradataml/context/context.py:481: TeradataMlRuntimeWarning: Warning: Password is URL encoded.
  warnings.warn("Warning: Password is URL encoded.", category=TeradataMlRuntimeWarning)


In [26]:
tdml.save_byom('logreg_iris_v1', "logreg_iris.onnx",'scikit_models')

Model is saved.


## Export Iris validation set for scoring

In [38]:
output = pd.DataFrame(X_test, columns = ['float_input_0','float_input_1','float_input_2','float_input_3'])

In [41]:
output.to_csv('Iris_features.csv',index=False, header=False, lineterminator='\n')

In [43]:
!head Iris_features.csv

5.7,2.5,5.0,2.0
5.2,4.1,1.5,0.1
6.1,2.8,4.0,1.3
4.5,2.3,1.3,0.3
6.5,2.8,4.6,1.5
5.0,3.0,1.6,0.2
6.4,2.8,5.6,2.2
5.0,3.4,1.6,0.4
5.6,2.9,3.6,1.3
6.9,3.1,5.4,2.1
